# Unstructured Search & Grover’s Algorithm — Detailed Notes (Session 5)
**Course:** CS490/5590 — Quantum Computing Applications in Data Science, AI, & Deep Learning  
**Instructor:** Luke Miller  
**Level:** Graduate / Advanced Undergraduate  

> **Purpose.**  This hand-out elaborates the slide bullets for Session 5.  It explains Grover’s algorithm from first principles, walks through oracle and diffusion operators, includes Qiskit code, shows how the quadratic speed-up is derived, and sketches data-science applications.  Mini-exercises with brief answers appear at the end.

---

## Session Road-map  

1. Recap of query-model speed-ups (DJ, BV)  
2. Unstructured-search problem statement  
3. Grover’s oracle + diffusion (one iteration)  
4. Multi-iteration amplitude amplification  
5. Handling multiple marked items; unknown $M$  
6. Complexity proofs & optimality  
7. Qiskit implementation (4-qubit demo)  
8. Applications & limitations in AI / DS  
9. Q&A  

---

## 0) Recap (Session 4 in one slide)

| Algorithm | Oracle type | Classical queries | Quantum queries |
|-----------|-------------|-------------------|-----------------|
| Deutsch–Jozsa | promise (const / balanced) | $2^{n-1}+1$ det. | **1** |
| Bernstein–Vazirani | linear parity | $n$ | **1** |
| Key idea | superposition + interference | | |

Grover generalises the idea to arbitrary *search* without promises and achieves **quadratic** speed-up: $O(N)\to O(\sqrt N)$.

---

## 1) The unstructured-search problem  

- Domain size: $N = 2^{\,n}$ (search space over $n$ qubits).  
- Oracle $f:\{0,1\}^n\to\{0,1\}$; at least one $x^\star$ s.t. $f(x^\star)=1$.  
- Goal: return some $x^\star$ with high probability, minimising the number of oracle calls.

Classical exhaustive search requires average $N/2$ queries; worst-case $N$.

---

## 2) Grover’s algorithm — conceptual outline  

1. **Initial superposition**  
   $$
     |s\rangle \;=\; H^{\otimes n}|0\rangle^{\!\otimes n} = \frac1{\sqrt N}\sum_{x=0}^{N-1}|x\rangle.
   $$
2. **Grover iteration** $G = D\,O_f$ composed of  
   - Oracle phase flip $O_f$ : $ (-1)^{f(x)}$ on basis $|x\rangle$  
   - Diffusion operator $D$ : reflection about $|s\rangle$  
3. **Repeat** $k \approx \bigl\lceil\frac{\pi}{4}\sqrt{N/M}\bigr\rceil$ times.  
4. **Measure**; output is a marked item with probability ≥ ~0.97 for correct $k$.

---

## 3) The oracle $O_f$

**Phase-oracle matrix**

$$
O_f = \sum_{x}( -1)^{f(x)} |x\rangle\langle x|.
$$

- Marked items pick up a $-1$ phase, unmarked remain $+1$.  
- Implemented via multi-controlled-$Z$; ancillas may be needed for reversible logic.  

> **Two-qubit example.** $f(11)=1$ else 0.  Oracle = `ccz` controlled on both qubits (or `ccx` + phase).

---

## 4) The diffusion operator $D$

Definition  

$$
D =  2|s\rangle\langle s| - I = H^{\otimes n}\,(2|0\rangle\langle0|-I)\,H^{\otimes n}.
$$

Circuit decomposition  

1. $H^{\otimes n}$  
2. Multi-qubit controlled-$Z$ that flips phase of $|0\rangle^{\!\otimes n}$.  
3. $H^{\otimes n}$.

**Geometric view** — Two consecutive reflections (about the “marked” hyperplane and about $|s\rangle$) rotate the state vector by angle $2\theta$ toward the solution subspace.

---

## 5) Amplitude amplification analysis  

Let  

$$
|w\rangle = \frac{1}{\sqrt M}\sum_{x:f(x)=1}|x\rangle, \quad
|r\rangle = \frac{1}{\sqrt{N-M}}\sum_{x:f(x)=0}|x\rangle.
$$

Express the state as  

$$
|s\rangle = \sin\theta\,|w\rangle + \cos\theta\,|r\rangle,
\quad \theta = \arcsin\sqrt{\frac{M}{N}}.
$$

Each Grover iteration performs rotation by $2\theta$ in the $\{|w\rangle,|r\rangle\}$ plane:

$$
G\bigl(\sin\phi\,|w\rangle + \cos\phi\,|r\rangle\bigr)
      = \sin(\phi+2\theta)\,|w\rangle + \cos(\phi+2\theta)\,|r\rangle.
$$

After $k$ iterations, success probability  
$$
P_k = \sin^2\!\bigl((2k+1)\theta\bigr).
$$

Choose $k = \left\lfloor\frac{\pi}{4\theta}-\tfrac12\right\rfloor \approx \frac{\pi}{4}\sqrt{N/M}$.

---

## 6) Optimality and lower bounds  

- **Grover optimal.** Bennett et al. (1997) prove $\Omega(\sqrt{N/M})$ is a lower bound for any quantum algorithm in the oracle model.  
- Classical deterministic/rand lower bound $\Omega(N/M)$.

---

## 7) Handling multiple or unknown solutions  

- If # solutions $M$ known: use $k_{\text{opt}}\approx\frac{\pi}{4}\sqrt{N/M}$.  
- If $M$ *unknown*:  
  - Use **quantum counting** (phase estimation on $G$) to estimate $\theta$, then apply Grover.  
  - Or iterate until success probability peaks, checking with intermediate measurements (incremental search).

---

## 8) Qiskit implementation example (n = 4, one target)

```python
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from math import pi, sqrt, floor

N = 16        # 4 qubits
marked = '1011'  # target state (little-endian)
n = 4
backend = AerSimulator()

# --- Oracle ---
oracle = QuantumCircuit(n)
# Flip phase of |marked>
for i, bit in enumerate(reversed(marked)):
    if bit == '0':
        oracle.x(i)
oracle.mcx(list(range(n-1)), n-1)        # multi-controlled Z via mcx
for i, bit in enumerate(reversed(marked)):
    if bit == '0':
        oracle.x(i)
oracle = oracle.to_gate(label='O_f')

# --- Diffusion ---
diff = QuantumCircuit(n)
diff.h(range(n))
diff.x(range(n))
diff.mcx(list(range(n-1)), n-1)
diff.x(range(n))
diff.h(range(n))
diff = diff.to_gate(label='D')

# --- Grover routine ---
grover = QuantumCircuit(n, n)
grover.h(range(n))               # uniform superposition

k = floor(pi/4*sqrt(N))          # optimal iterations for M=1
for _ in range(k):
    grover.append(oracle, range(n))
    grover.append(diff,   range(n))

grover.measure(range(n), range(n))
print(grover.draw('text'))

job = backend.run(transpile(grover, backend), shots=1024)
print(job.result().get_counts())
```

For ideal simulation you should see `'1011'` ~ 95 % of the time.

---

## 9) Applications in AI / data science  

| Use‐case | How Grover helps | Caveats |
|----------|------------------|---------|
| Fraud / anomaly detection | Amplify rare “bad” patterns flagged by oracle predicate. | Need efficient oracle circuit encoding pattern. |
| Feature selection | Oracle checks if subset meets accuracy ≥ τ; Grover explores $2^n$ subsets. | Costly oracle = full model eval. |
| Combinatorial optimisation | Oracle marks states with cost ≤ bound; Grover used iteratively (“fixed-target search”). | Often combined with classical heuristics or QAOA. |
| Quantum ML kernels | Amplitude amplification subroutine in QSVM to skew sampling toward support vectors. | Still theoretical on NISQ scale. |

---

## 10) Practical limitations  

1. **Oracle construction cost** may scale like $O(\text{poly}(n))$ gates; if large, classical pre-processing may dominate.  
2. **Noise sensitivity.** Each Grover iteration adds depth; error grows with $\sqrt N$. Error-mitigation required on NISQ.  
3. **Quadratic ≠ exponential.** For very large unstructured domains the residual $\sqrt N$ may still be enormous.

---

## 11) Worked algebra / proofs  

**A — Rotation proof.** Show that $D|s\rangle=|s\rangle$ and $D|w\rangle = -(1 - 2/N)|w\rangle - (2\sqrt{(N-M)M}/N)|r\rangle$.  Combine with oracle to derive 2θ rotation.  

**B — Optimal k.** Set derivative of $P_k = \sin^2((2k+1)\theta)$ to zero ⇒ $(2k+1)\theta ≈ \frac{\pi}{2}$.  

---

## 12) Mini-exercises (answers in Appendix)  

1. Implement Grover with $N=8$ (three qubits) and target `010`; verify 2 iterations maximise success.  
2. Prove diffusion operator equals $2|s\rangle\langle s| - I$.  
3. If $M=2$ targets in $N=16$, compute optimal $k$.  
4. Explain why one extra iteration after optimal $k$ *reduces* success probability.  
5. Outline quantum counting circuit for estimating $M$.  

---

## 13) FAQ / common pitfalls  

- **“Why the aux qubit in some textbooks?”** Early versions use bit-flip oracles with $|-\rangle$ ancilla; phase oracle is equivalent and avoids extra qubit.  
- **“My simulator shows other states.”** Too many/few iterations; verify $k$. Check endianness (`qiskit` prints basis as c_{n-1}…c_0).  
- **“Multi-controlled gates won’t transpile on hardware.”** Need ancilla-enabled decomposition or iterative Grover with ancilla reuse.  
- **“Can Grover index real SQL tables?”** Only if an efficient coherent oracle prepares the table in superposition—current hardware cannot stream petabytes coherently.

---

## 14) Summary (Session 5)

- Grover provides **quadratic** query speed-up for unstructured search.  
- Core components: phase oracle $O_f$ + diffusion $D$.  
- After $\sim \frac{\pi}{4}\sqrt{N/M}$ iterations, measuring yields a marked item with high probability.  
- Speed-up is optimal in oracle model; practical benefit hinges on constructing shallow, low-error oracles.  
- Grover underpins amplitude-amplification frameworks and influences quantum optimisation and ML algorithms.

---

## 15) Looking ahead  

- **Next Session:** Quantum Noise & Transpilation — mitigating errors and mapping circuits to hardware.  
- **Homework 2:** derive Grover amplitude formula, implement 3-qubit example, and explore multiple-solution case.

---

## Appendix — solutions to mini-exercises (sketch)

1. $N=8, M=1 \Rightarrow k\approx\left\lfloor \frac{\pi}{4}\sqrt8 - \tfrac12\right\rfloor = 2$. Simulator shows target ≥ 95 %.  
2. Expand $D$ definition using $H^{\otimes n}$ and verify matrix elements.  
3. $k\approx \frac{\pi}{4}\sqrt{16/2} ≈ \left\lfloor 2.8\right\rfloor=2$.  
4. Rotation picture: state passes optimal angle; probability oscillates back down (over-rotation).  
5. Quantum counting = apply phase estimation to $G$; eigen-phases encode $\theta$, hence $M$.

